# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import re
import pickle
import numpy as np
import pandas as pd
from itertools import chain

import sqlalchemy
from sqlalchemy import create_engine

import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
nltk.download(['stopwords','wordnet','punkt','averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Support Vector Machines
from sklearn.svm import SVC
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
# Logistic Regression
from sklearn.linear_model import LogisticRegression
# KNeighbor Classifier - K-Nearest Neighbors (KNN)
from sklearn.neighbors import KNeighborsClassifier
# Naive Bayes
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("Message", engine)
df = df.sample(frac=0.25)
X = df['message'].values
y = df.drop(['id', 'message', 'original', 'genre'], axis=1).values

In [ ]:
print(f"df type: {type(df)}\nLen of df: {len(df)}")
print(f"Shape of X: {X.shape},\nShape of Y: {y.shape}")
print(list(df.columns))

In [ ]:
# np.sum(df.isnull())

# # Check nulls
# 100 * (df.isnull().mean())

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    # Tokenize text into words
    words = list(chain.from_iterable([word_tokenize(t) for t in text]))
    
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words("english")]
    
    # Tag words with POS
    tagged_words = pos_tag(words)
    
    # Initiate lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    # Iterate through each tagged word
    for word, tag in tagged_words:
        # Lemmatize based on POS tag
        # if tag.startswith('N'):
        #     clean_tok = lemmatizer.lemmatize(word, pos='n').lower().strip()
        # elif tag.startswith('V'):
        #     clean_tok = lemmatizer.lemmatize(word, pos='v').lower().strip()
        # elif tag.startswith('J'):
        #     clean_tok = lemmatizer.lemmatize(word, pos='a').lower().strip()
        # else:
        #     clean_tok = lemmatizer.lemmatize(word).lower().strip()
        clean_tok = lemmatizer.lemmatize(word).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

In [ ]:
def tokenize1(text):
    words = list(chain.from_iterable([word_tokenize(t) for t in text]))
    stop_words = set(stopwords.words("english"))
    tagged_words = pos_tag(words)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = [lemmatizer.lemmatize(word).lower().strip() for word, tag in tagged_words if word not in stop_words]
    return clean_tokens

In [ ]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenize)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

In [ ]:
display_results(y_test, y_pred)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# Iterate through each output category and calculate the F1 score, precision, and recall
for i, column in enumerate(y.columns):
    print(f"Category: {column}\n")
    print(classification_report(y_test[column], y_pred[:, i]))
    print("------------------------")

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
def build_model():
    pipeline = Pipeline([
        # ('vect',CountVectorizer(tokenizer=tokenize1)),
        # ('tfidf', TfidfTransformer()),
        ('vect', TfidfVectorizer(tokenizer=tokenize)),
        ('clf', RandomForestClassifier())
    ])
    # specify parameters for grid search
    parameters = {
        "vect__ngram_range":[(1,1),(1,2)],
        "clf__n_estimators": [10,50,100],
        "clf__min_samples_split":[2, 3, 4],
        "clf__criterion":["gini", "entropy"]
    }

    # create grid search object
    cv = GridSearchCV(pipeline, param_grid=parameters)

    return cv

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

In [ ]:
model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
display_results(y_test, y_pred)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# Iterate through each output category and calculate the F1 score, precision, and recall
for i, column in enumerate(y.columns):
    print(f"Category: {column}\n")
    print(classification_report(y_test[column], y_pred[:, i]))
    print("------------------------")

In [ ]:
# Export model to pickle file

with open('rfc_model.pkl', 'wb') as file:
    pickle.dump(model, file)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# def build_model():
#     models = []

#     pipeline_rf = Pipeline([
#         ('vect', CountVectorizer(tokenizer=tokenize)),
#         ('tfidf', TfidfTransformer()),
#         ('clf', MultiOutputClassifier(RandomForestClassifier()))
#     ])
    
#     parameters_rf = {
#         'vect__ngram_range': [(1, 1), (1, 2)],
#         'clf__estimator__n_estimators': [10, 50, 100],
#         'clf__estimator__min_samples_split': [2, 3, 4],
#         # 'clf__estimator__criterion': ['gini', 'entropy']
#     }
    
#     cv_rf = GridSearchCV(pipeline_rf, param_grid=parameters_rf)
#     models.append(('RandomForestClassifier', cv_rf))

    # pipeline_lr = Pipeline([
    #     ('vect', CountVectorizer(tokenizer=tokenize)),
    #     ('tfidf', TfidfTransformer()),
    #     ('clf', MultiOutputClassifier(LogisticRegression()))
    # ])    
    
    # parameters_lr = {
    #     "vect__ngram_range": [(1, 1), (1, 2)],
    #     "clf__estimator__C": [0.1, 1, 10],
    #     # "clf__estimator__solver": ["lbfgs", "liblinear"]
    # }
    
    # cv_lr = GridSearchCV(pipeline_lr, param_grid=parameters_lr)
    # models.append(('LogisticRegression', cv_lr))
    
    # pipeline_svc = Pipeline([
    #     ('vect', CountVectorizer(tokenizer=tokenize)),
    #     ('tfidf', TfidfTransformer()),
    #     ('clf', MultiOutputClassifier(SVC()))
    # ])
    
    # parameters_svc = {
    #     'vect__ngram_range': [(1, 1), (1, 2)],
    #     'clf__estimator__C': [1, 10],
    #     'clf__estimator__kernel': ['linear', 'rbf']
    # }
    
    # cv_svc = GridSearchCV(pipeline_svc, param_grid=parameters_svc)
    # models.append(('SVC', cv_svc))
    
    # pipeline_gb = Pipeline([
    #     ('vect', CountVectorizer(tokenizer=tokenize)),
    #     ('tfidf', TfidfTransformer()),
    #     ('clf', MultiOutputClassifier(GradientBoostingClassifier()))
    # ])
    
    # parameters_gb = {
    #     'vect__ngram_range': [(1, 1), (1, 2)],
    #     'clf__estimator__n_estimators': [50, 100],
    #     'clf__estimator__learning_rate': [0.1, 0.5]
    # }
    
    # cv_gb = GridSearchCV(pipeline_gb, param_grid=parameters_gb)
    # models.append(('GradientBoosting', cv_gb))
    
    # pipeline_kneigh = Pipeline([
    #     ('vect', CountVectorizer(tokenizer=tokenize)),
    #     ('tfidf', TfidfTransformer()),
    #     ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    # ])
    
    # parameters_kneigh = {
    #     'vect__ngram_range': [(1, 1), (1, 2)],
    #     'clf__estimator__n_neighbors': [3, 5, 7],
    #     # 'clf__estimator__weights': ['uniform', 'distance']
    # }
    
    # cv_kneigh = GridSearchCV(pipeline_kneigh, param_grid=parameters_kneigh)
    # models.append(('KNeighborsClassifier', cv_kneigh))
    
    # pipeline_naive = Pipeline([
    #     ('vect', CountVectorizer(tokenizer=tokenize)),
    #     ('tfidf', TfidfTransformer()),
    #     ('clf', MultiOutputClassifier(MultinomialNB()))
    # ])
    
    # parameters_naive = {
    #     'vect__ngram_range': [(1, 1), (1, 2)],
    #     'clf__estimator__alpha': [0.5, 1.0]
    # }
    
    # cv_naive = GridSearchCV(pipeline_naive, param_grid=parameters_naive)
    # models.append(('MultinomialNB', cv_naive))

    # return models

In [ ]:
# models = build_model()
# for model_name, model in models:
#     print(f"Training {model_name}...")
#     model.fit(X_train, y_train)
#     print(f"Evaluating {model_name}...")
#     y_pred = model.predict(X_test)
#     print(f"Classification Report for {model_name}:")
#     print(classification_report(y_test, y_pred))
#     print("----------------------------------------------------")

### 9. Export your model as a pickle file

In [ ]:
# models = build_model()
# # Train and evaluate models
# best_model = None
# best_model_name = None
# best_model_score = 0

# for model_name, model in models:
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     score = classification_report(y_test, y_pred)
    
#     if score > best_model_score:
#         best_model_score = score
#         best_model = model
#         best_model_name = model_name

# # Save the best model as a pickle file
# with open('best_model.pkl', 'wb') as f:
#     pickle.dump(best_model, f)

# print(best_model_name)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.